In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

# import torch
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# Good data
# df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_with_linguistics.csv')

In [3]:
# Messy data
# df = pd.read_csv('./data/affcon_final_politeness_strategies_merge.csv')

In [4]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')

In [5]:
df = df.dropna()

In [6]:
# Throughput
df_throughput = df[['Throughput.1', 'Throughput.2', 'Throughput.3', 'Throughput.4', 'Throughput.5']].copy()
df_throughput['avg_throughput'] = df_throughput.mean(axis=1)
df_throughput['avg_throughput'] = df_throughput['avg_throughput'] / df_throughput['avg_throughput'].max()

# WorkTime
df_worktime = df[['WorkTime.1', 'WorkTime.2', 'WorkTime.3', 'WorkTime.4', 'WorkTime.5']].copy()
df_worktime['avg_worktime'] = df_worktime.mean(axis=1)
df_worktime['avg_worktime'] = df_worktime['avg_worktime'] / df_worktime['avg_worktime'].max()

In [7]:
indices = range(df.shape[0])

### Individual Classifiers are LSTMS

In [8]:
train, test, indices_train, indices_test = train_test_split(df, indices, test_size=0.2)

new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].tolist()
y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()

X_train_col = train['Input.full_text']

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_test_col = test['Input.full_text']

In [9]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [10]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [11]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

# tok.fit_on_texts(X_test_col)
X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

In [12]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

### Construct individual LSTM models

In [13]:
# Rapport model
rapport_model = create_lstm()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
rapport_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_rapport), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 4s 51ms/step - loss: 0.4288 - accuracy: 0.8520 - f1_m: 0.9153 - recall_m: 0.9870 - precision_m: 0.8640 - val_loss: 0.4262 - val_accuracy: 0.8628 - val_f1_m: 0.9259 - val_recall_m: 1.0000 - val_precision_m: 0.8625
Epoch 2/15
72/72 [==============================] - 3s 44ms/step - loss: 0.4049 - accuracy: 0.8625 - f1_m: 0.9269 - recall_m: 1.0000 - precision_m: 0.8644 - val_loss: 0.4295 - val_accuracy: 0.8628 - val_f1_m: 0.9259 - val_recall_m: 1.0000 - val_precision_m: 0.8625


In [14]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

In [15]:
rapport_pred_test_round = rapport_pred_test.round()
precision_recall_fscore_support(y_test_rapport, rapport_pred_test_round, average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4313984168865435, 0.5, 0.4631728045325779, None)

In [16]:
# Game move model
gamemove_model = create_lstm()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
gamemove_model.fit(X_train,y_train_gamemove,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_gamemove), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 3s 48ms/step - loss: 0.2937 - accuracy: 0.9190 - f1_m: 0.9524 - recall_m: 0.9887 - precision_m: 0.9267 - val_loss: 0.3171 - val_accuracy: 0.9393 - val_f1_m: 0.9685 - val_recall_m: 1.0000 - val_precision_m: 0.9392
Epoch 2/15
72/72 [==============================] - 3s 45ms/step - loss: 0.2604 - accuracy: 0.9286 - f1_m: 0.9615 - recall_m: 1.0000 - precision_m: 0.9262 - val_loss: 0.3832 - val_accuracy: 0.9393 - val_f1_m: 0.9685 - val_recall_m: 1.0000 - val_precision_m: 0.9392


In [17]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

In [18]:
gamemove_pred_test_round = gamemove_pred_test.round()
precision_recall_fscore_support(y_test_gamemove, gamemove_pred_test_round, average='macro')

(0.46965699208443273, 0.5, 0.48435374149659866, None)

In [19]:
# Reasoning model
reasoning_model = create_lstm()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
reasoning_model.fit(X_train,y_train_reasoning,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_reasoning), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 4s 51ms/step - loss: 0.4882 - accuracy: 0.8224 - f1_m: 0.9027 - recall_m: 0.9934 - precision_m: 0.8295 - val_loss: 0.4710 - val_accuracy: 0.8360 - val_f1_m: 0.9097 - val_recall_m: 1.0000 - val_precision_m: 0.8352
Epoch 2/15
72/72 [==============================] - 3s 48ms/step - loss: 0.4613 - accuracy: 0.8275 - f1_m: 0.9045 - recall_m: 1.0000 - precision_m: 0.8265 - val_loss: 0.4582 - val_accuracy: 0.8360 - val_f1_m: 0.9097 - val_recall_m: 1.0000 - val_precision_m: 0.8352
Epoch 3/15
72/72 [==============================] - 3s 46ms/step - loss: 0.4503 - accuracy: 0.8275 - f1_m: 0.9065 - recall_m: 1.0000 - precision_m: 0.8299 - val_loss: 0.4650 - val_accuracy: 0.8360 - val_f1_m: 0.9097 - val_recall_m: 1.0000 - val_precision_m: 0.8352


In [20]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

In [21]:
reasoning_pred_test_round = reasoning_pred_test.round()
precision_recall_fscore_support(y_test_reasoning, reasoning_pred_test_round, average='macro')

(0.41798592788038696, 0.5, 0.45532934131736524, None)

In [22]:
# Share Information model
shareinfo_model = create_lstm()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_share_information,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_share_information), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 4s 50ms/step - loss: 0.4605 - accuracy: 0.8382 - f1_m: 0.9127 - recall_m: 0.9974 - precision_m: 0.8421 - val_loss: 0.4463 - val_accuracy: 0.8421 - val_f1_m: 0.9137 - val_recall_m: 1.0000 - val_precision_m: 0.8415
Epoch 2/15
72/72 [==============================] - 3s 46ms/step - loss: 0.4435 - accuracy: 0.8402 - f1_m: 0.9140 - recall_m: 1.0000 - precision_m: 0.8423 - val_loss: 0.4387 - val_accuracy: 0.8421 - val_f1_m: 0.9137 - val_recall_m: 1.0000 - val_precision_m: 0.8415
Epoch 3/15
72/72 [==============================] - 3s 45ms/step - loss: 0.4338 - accuracy: 0.8402 - f1_m: 0.9139 - recall_m: 1.0000 - precision_m: 0.8423 - val_loss: 0.4437 - val_accuracy: 0.8421 - val_f1_m: 0.9137 - val_recall_m: 1.0000 - val_precision_m: 0.8415


In [23]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

In [24]:
shareinfo_pred_test_round = shareinfo_pred_test.round()
precision_recall_fscore_support(y_test_share_information, shareinfo_pred_test_round, average='macro')

(0.4210642040457344, 0.5, 0.45714967772738124, None)

In [25]:
# Deception model
deception_model = create_lstm()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
deception_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 4s 51ms/step - loss: 0.2264 - accuracy: 0.9458 - f1_m: 0.9711 - recall_m: 0.9925 - precision_m: 0.9535 - val_loss: 0.2185 - val_accuracy: 0.9442 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9438
Epoch 2/15
72/72 [==============================] - 3s 45ms/step - loss: 0.1960 - accuracy: 0.9526 - f1_m: 0.9759 - recall_m: 1.0000 - precision_m: 0.9532 - val_loss: 0.2193 - val_accuracy: 0.9442 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9438


In [26]:
deception_pred = deception_model.predict(X_train)
deception_pred_test = deception_model.predict(X_test)
deception_pred_test_round = deception_pred_test.round()
precision_recall_fscore_support(y_test_deception, deception_pred_test_round, average='macro')

(0.47207563764292, 0.5, 0.4856367337706401, None)

### One hot encodings

In [27]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred[i][0]
    pred_obj_1['reasoning'] = reasoning_pred[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred[i][0]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred[i][0]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [28]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred_test[i][0]
    pred_obj_1['reasoning'] = reasoning_pred_test[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred_test[i][0]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

In [29]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(2, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [30]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = create_joint_model(pred_df_full)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df_full,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception
Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 6ms/step - loss: 0.6956 - acc: 0.0775 - f1_m: 0.9757 - precision_m: 0.9529 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.3091 - val_

In [31]:
joint_predict = joint_full_model.predict(pred_test_df_full)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.49590957513472866, 0.4808485746454492, 0.3480497164107832, None)

In [32]:
print('Joint full model with one hot encoding, predicting rapport')
joint_full_model = create_joint_model(pred_df)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport
Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 4ms/step - loss: 0.7418 - acc: 0.1375 - f1_m: 0.9260 - precision_m: 0.8634 - recall_m: 1.0000 - val_loss: 0.6957 - val_acc: 0.1372 - val_f1

143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.4405 - f1_m: 0.9252 - precision_m: 0.8618 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.4956 - val_f1_m: 0.9254 - val_precision_m: 0.8619 - val_recall_m: 1.0000
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.4359 - f1_m: 0.9240 - precision_m: 0.8601 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.4600 - val_f1_m: 0.9254 - val_precision_m: 0.8619 - val_recall_m: 1.0000
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.4382 - f1_m: 0.9260 - precision_m: 0.8634 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.4604 - val_f1_m: 0.9254 - val_precision_m: 0.8619 - val_recall_m: 1.0000
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.4387 - f1_m: 0.9251 - precision_m: 0.8618 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.4635 - val_f1_m: 0.9254 - val_precision_m: 0.8619 - val_r

In [33]:
joint_predict = joint_full_model.predict(pred_test_df)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5047228804634019, 0.5098457225750803, 0.4080475221159632, None)

### Joint Model by Keras Concatenate

In [34]:
def create_concatenate_keras_model(models_arr):
    commonInput = Input(shape=[max_len])

    input_model_arr = []
    for model in models_arr: 
        outmodel = model(commonInput)
        input_model_arr.append(outmodel)
    
    mergedOut = Concatenate()(input_model_arr)

    mergedOut = Flatten()(mergedOut)    
    mergedOut = Dense(256, activation='relu')(mergedOut)
    mergedOut = Dropout(.5)(mergedOut)
    mergedOut = Dense(128, activation='relu')(mergedOut)
    mergedOut = Dropout(.35)(mergedOut)
    mergedOut = Dense(2, activation='softmax')(mergedOut)  #Cuz binary

    mergedModel = Model(commonInput, mergedOut)
    mergedModel.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', f1_m, recall_m, precision_m])
    
    return mergedModel

In [35]:
print('Joint model by concatenate, predicting deception')
merged_model = create_concatenate_keras_model([gamemove_model, reasoning_model, shareinfo_model, rapport_model])
merged_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test,y_test_deception), callbacks=[early_stop])

Joint model by concatenate, predicting deception
Epoch 1/15
72/72 [==============================] - 13s 186ms/step - loss: 0.6938 - accuracy: 0.5180 - f1_m: 0.9759 - recall_m: 1.0000 - precision_m: 0.9532 - val_loss: 0.6931 - val_accuracy: 0.9442 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9438
Epoch 2/15
72/72 [==============================] - 12s 171ms/step - loss: 0.6931 - accuracy: 0.5114 - f1_m: 0.9760 - recall_m: 1.0000 - precision_m: 0.9532 - val_loss: 0.6931 - val_accuracy: 0.0558 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9438


In [36]:
print('Joint model by concatenate, predicting rapport')
merged_model = create_concatenate_keras_model([gamemove_model, reasoning_model, shareinfo_model])
merged_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test,y_test_rapport), callbacks=[early_stop])

Joint model by concatenate, predicting rapport
Epoch 1/15
72/72 [==============================] - 10s 138ms/step - loss: 0.6932 - accuracy: 0.5030 - f1_m: 0.9269 - recall_m: 1.0000 - precision_m: 0.8644 - val_loss: 0.6931 - val_accuracy: 0.1372 - val_f1_m: 0.9259 - val_recall_m: 1.0000 - val_precision_m: 0.8625
Epoch 2/15
72/72 [==============================] - 9s 125ms/step - loss: 0.6931 - accuracy: 0.5018 - f1_m: 0.9269 - recall_m: 1.0000 - precision_m: 0.8644 - val_loss: 0.6931 - val_accuracy: 0.1372 - val_f1_m: 0.9259 - val_recall_m: 1.0000 - val_precision_m: 0.8625


## Weighted against Throughput & WorkTime

In [37]:
# Train weighted encodings
pred_df_full_throughput = pred_df_full.copy()
pred_df_full_worktime = pred_df_full.copy()
pred_df_throughput = pred_df.copy()
pred_df_worktime = pred_df.copy()

throughput_values = df_throughput['avg_throughput'].take(indices_train).values
pred_df_full_throughput = pred_df_full_throughput.mul(throughput_values, axis=0)
pred_df_throughput = pred_df_throughput.mul(throughput_values, axis=0)

worktime_values = df_worktime['avg_worktime'].take(indices_train).values
pred_df_full_worktime = pred_df_full_worktime.mul(worktime_values, axis=0)
pred_df_worktime = pred_df_worktime.mul(worktime_values, axis=0)

In [38]:
# Test weighted encodings
pred_df_full_throughput_test = pred_test_df_full.copy()
pred_df_full_worktime_test = pred_test_df_full.copy()
pred_df_throughput_test = pred_test_df.copy()
pred_df_worktime_test = pred_test_df.copy()

throughput_values_test = df_throughput['avg_throughput'].take(indices_test).values
pred_df_full_throughput_test = pred_df_full_throughput_test.mul(throughput_values_test, axis=0)
pred_df_throughput_test = pred_df_throughput_test.mul(throughput_values_test, axis=0)

worktime_values_test = df_worktime['avg_worktime'].take(indices_test).values
pred_df_full_worktime_test = pred_df_full_worktime_test.mul(worktime_values_test, axis=0)
pred_df_worktime_test = pred_df_worktime_test.mul(worktime_values_test, axis=0)

## Throughput only

### Joint Model with one hot encoding 

In [39]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(2, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [40]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = create_joint_model(pred_df_full_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_throughput, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_full_throughput_test,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6932 - acc: 0.5114 - f1_m: 0.9757 - precision_m: 0.9529 - recall_m: 1.0000 - val_loss: 0.6932 

In [41]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.49036536536536535, 0.495764094928283, 0.49078409517893495, None)

In [42]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [43]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = create_joint_model(pred_df_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_throughput, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_throughput_test,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6937 - acc: 0.4946 - f1_m: 0.9261 - precision_m: 0.8634 - recall_m: 1.0000 - val_loss: 0.6933 - 

In [44]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.4197173769043939, 0.33368619148435663, 0.31176256949424025, None)

## WorkTime only

### Joint Model with one hot encoding 

In [45]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(2, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [46]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = create_joint_model(pred_df_full_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_worktime, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_full_worktime_test,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6931 - acc: 0.4764 - f1_m: 0.9755 - precision_m: 0.9529 - recall_m: 0.9996 - val_loss: 0.6931 

In [47]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)


In [48]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = create_joint_model(pred_df_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_worktime, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_worktime_test,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6931 - acc: 0.1375 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_los